# load drive and librabies


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyod

In [ ]:
%cd /content/drive/MyDrive/MQ-solutions/ood_practical/RTL/CIFAR/CIFAR

/content/drive/.shortcut-targets-by-id/19rDR7CIzlqpEM2y_t3vRDkf0-aHH8tdE/ood_practical/RTL/CIFAR/CIFAR


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import os

import pandas as pd
import sys

import torch.backends.cudnn as cudnn
import torchvision.transforms as trn
import torchvision.datasets as dset
import torch.nn.functional as F
from models.wrn import WideResNet
from skimage.filters import gaussian as gblur
from PIL import Image as PILImage
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CIFAR10Dataset

In [ ]:
# 1. Chuẩn bị dữ liệu
class CIFAR10Dataset(Dataset):
    def __init__(self, data_files, transform=None):
        self.data = []
        self.labels = []
        self.transform = transform

        for file in data_files:
            with open(file, 'rb') as f:
                batch = pickle.load(f, encoding='bytes')
                self.data.append(batch[b'data'])
                self.labels.extend(batch[b'labels'])

        self.data = np.vstack(self.data).reshape(-1, 3, 32, 32).astype(np.uint8)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.data[idx].transpose(1, 2, 0)
        img = Image.fromarray(img)
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

# Đường dẫn đến các file batch
data_dir = "/content/drive/MyDrive/MQ-solutions/ood_practical/RTL/CIFAR/CIFAR/data/cifar10py/cifar-10-batches-py"

train_files = [
    f"{data_dir}/data_batch_1",
    f"{data_dir}/data_batch_2",
    f"{data_dir}/data_batch_3",
    f"{data_dir}/data_batch_4",
    f"{data_dir}/data_batch_5"
]
test_file = f"{data_dir}/test_batch"

# 2. Transforms cho hình ảnh
mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

transform = trn.Compose([
    trn.Resize(224),
    trn.ToTensor(),
    trn.Normalize(mean, std)
])

# Tạo dataset và dataloader
train_dataset = CIFAR10Dataset(train_files, transform=transform)
test_dataset = CIFAR10Dataset([test_file], transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
vits16 = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
vits16.to(device)

Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_deitsmall16_pretrain.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 89.5MB/s]


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
  (head): Identity()
)

# extract features

In [ ]:
def extract_features(model, loader, name):
  features = []
  vits16.eval()

  with torch.no_grad():  # Disable gradient calculation during inference
    for batch_idx, (data, target) in enumerate(tqdm(loader, desc="Processing")):  # Use tqdm for progress bar
        # Chuyển dữ liệu sang thiết bị (CPU or GPU)
        data = data.to(device)  # Replace 'device' with your device (e.g., 'cuda' or 'cpu')
        # Tính toán output của mô hình
        output = vits16(data)
        # Lưu output vào list
        features.append(output)

  features = torch.cat(features, dim=0)
  torch.save(features, f'/content/drive/MyDrive/MQ-solutions/ood_practical/RTL/CIFAR/CIFAR/features224/{name}.pt')

In [ ]:
extract_features(vits16, train_loader, 'train_features')

Processing: 100%|██████████| 782/782 [03:01<00:00,  4.31it/s]


In [ ]:
extract_features(vits16, test_loader, 'test_features')

Processing: 100%|██████████| 157/157 [00:37<00:00,  4.16it/s]


## extract ood features

In [ ]:
# /////////////// Textures ///////////////
ood_data = dset.ImageFolder(root="./data/dtd/images/",
                        transform=trn.Compose([trn.Resize(224), trn.CenterCrop(32),
                                               trn.ToTensor(), trn.Normalize(mean, std)]))
ood_loader = torch.utils.data.DataLoader(ood_data, batch_size=64, shuffle=True,
                                     num_workers=0, pin_memory=False)
ood_loader.name = "Textures"
print('\n\nTexture Detection')

extract_features(vits16, ood_loader, 'texture_features')



Texture Detection


Processing: 100%|██████████| 89/89 [47:16<00:00, 31.87s/it]


In [ ]:
# /////////////// Places365 ///////////////
ood_data = dset.ImageFolder(root="./data/Places365/",
                            transform=trn.Compose([trn.Resize(224), trn.CenterCrop(32),
                                                   trn.ToTensor(), trn.Normalize(mean, std)]))
ood_loader = torch.utils.data.DataLoader(ood_data, batch_size=64, shuffle=True,
                                         num_workers=0, pin_memory=False)
ood_loader.name = "Places365"
print('\n\nPlaces365 Detection')
places365_features = extract_features(vits16, ood_loader, 'places365_features')



Places365 Detection


Processing: 100%|██████████| 303/303 [13:34<00:00,  2.69s/it]


In [ ]:
# /////////////// LSUN-C ///////////////
ood_data = dset.ImageFolder(root="./data/LSUN/",
                            transform=trn.Compose([trn.Resize(224), trn.ToTensor(), trn.Normalize(mean, std)]))
ood_loader = torch.utils.data.DataLoader(ood_data, batch_size=64, shuffle=True,
                                         num_workers=0, pin_memory=False)
ood_loader.name = "LSUN-C"
print('\n\nLSUN_C Detection')
extract_features(vits16, ood_loader, 'lsun-c_features')

In [ ]:
# /////////////// LSUN-R ///////////////
ood_data = dset.ImageFolder(root="./data/LSUN_resize/",
                            transform=trn.Compose([trn.Resize(224), trn.ToTensor(), trn.Normalize(mean, std)]))
ood_loader = torch.utils.data.DataLoader(ood_data, batch_size=64, shuffle=True,
                                         num_workers=0, pin_memory=False)
ood_loader.name = "LSUN-R"
print('\n\nLSUN_Resize Detection')
extract_features(vits16, ood_loader, 'lsun-r_features')

In [ ]:
# /////////////// iSUN ///////////////
ood_data = dset.ImageFolder(root="./data/iSUN/",
                            transform=trn.Compose([trn.Resize(224), trn.ToTensor(), trn.Normalize(mean, std)]))
ood_loader = torch.utils.data.DataLoader(ood_data, batch_size=64, shuffle=True,
                                         num_workers=0, pin_memory=False)
ood_loader.name = "iSUN"
print('\n\niSUN Detection')
extract_features(vits16, ood_loader, 'isun_features')

### SVHN


In [ ]:
%cd /content/drive/MyDrive/MQ-solutions/ood_practical/RTL/CIFAR

In [ ]:
import torch.utils.data as data
from PIL import Image
import os
import os.path
import numpy as np


class SVHN(data.Dataset):
    url = ""
    filename = ""
    file_md5 = ""

    split_list = {
        'train': ["http://ufldl.stanford.edu/housenumbers/train_32x32.mat",
                  "train_32x32.mat", "e26dedcc434d2e4c54c9b2d4a06d8373"],
        'test': ["http://ufldl.stanford.edu/housenumbers/test_32x32.mat",
                 "test_32x32.mat", "eb5a983be6a315427106f1b164d9cef3"],
        'extra': ["http://ufldl.stanford.edu/housenumbers/extra_32x32.mat",
                  "extra_32x32.mat", "a93ce644f1a588dc4d68dda5feec44a7"],
        'train_and_extra': [
                ["http://ufldl.stanford.edu/housenumbers/train_32x32.mat",
                 "train_32x32.mat", "e26dedcc434d2e4c54c9b2d4a06d8373"],
                ["http://ufldl.stanford.edu/housenumbers/extra_32x32.mat",
                 "extra_32x32.mat", "a93ce644f1a588dc4d68dda5feec44a7"]]}

    def __init__(self, root, split='train',
                 transform=None, target_transform=None, download=False):
        self.root = root
        self.transform = transform
        self.target_transform = target_transform
        self.split = split  # training set or test set or extra set

        if self.split not in self.split_list:
            raise ValueError('Wrong split entered! Please use split="train" '
                             'or split="extra" or split="test" '
                             'or split="train_and_extra" ')

        if self.split == "train_and_extra":
            self.url = self.split_list[split][0][0]
            self.filename = self.split_list[split][0][1]
            self.file_md5 = self.split_list[split][0][2]
        else:
            self.url = self.split_list[split][0]
            self.filename = self.split_list[split][1]
            self.file_md5 = self.split_list[split][2]

        # import here rather than at top of file because this is
        # an optional dependency for torchvision
        import scipy.io as sio

        # reading(loading) mat file as array
        loaded_mat = sio.loadmat(os.path.join(root, self.filename))

        if self.split == "test":
            self.data = loaded_mat['X']
            self.targets = loaded_mat['y']
            # Note label 10 == 0 so modulo operator required
            self.targets = (self.targets % 10).squeeze()    # convert to zero-based indexing
            self.data = np.transpose(self.data, (3, 2, 0, 1))
        else:
            self.data = loaded_mat['X']
            self.targets = loaded_mat['y']

            if self.split == "train_and_extra":
                extra_filename = self.split_list[split][1][1]
                loaded_mat = sio.loadmat(os.path.join(root, extra_filename))
                self.data = np.concatenate([self.data,
                                                  loaded_mat['X']], axis=3)
                self.targets = np.vstack((self.targets,
                                               loaded_mat['y']))
            # Note label 10 == 0 so modulo operator required
            self.targets = (self.targets % 10).squeeze()    # convert to zero-based indexing
            self.data = np.transpose(self.data, (3, 2, 0, 1))

    def __getitem__(self, index):
        if self.split == "test":
            img, target = self.data[index], self.targets[index]
        else:
            img, target = self.data[index], self.targets[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(np.transpose(img, (1, 2, 0)))

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        if self.split == "test":
            return len(self.data)
        else:
            return len(self.data)

    def _check_integrity(self):
        root = self.root
        if self.split == "train_and_extra":
            md5 = self.split_list[self.split][0][2]
            fpath = os.path.join(root, self.filename)
            train_integrity = check_integrity(fpath, md5)
            extra_filename = self.split_list[self.split][1][1]
            md5 = self.split_list[self.split][1][2]
            fpath = os.path.join(root, extra_filename)
            return check_integrity(fpath, md5) and train_integrity
        else:
            md5 = self.split_list[self.split][2]
            fpath = os.path.join(root, self.filename)
            return check_integrity(fpath, md5)

    def download(self):
        if self.split == "train_and_extra":
            md5 = self.split_list[self.split][0][2]
            download_url(self.url, self.root, self.filename, md5)
            extra_filename = self.split_list[self.split][1][1]
            md5 = self.split_list[self.split][1][2]
            download_url(self.url, self.root, extra_filename, md5)
        else:
            md5 = self.split_list[self.split][2]
            download_url(self.url, self.root, self.filename, md5)


In [ ]:
# # /////////////// SVHN /////////////// # cropped and no sampling of the test set
# ood_data = SVHN(root='./CIFAR/data/SVHN/', split="test",
#                      transform=trn.Compose(
#                          [trn.Resize(224),
#                          trn.ToTensor(), trn.Normalize(mean, std)]), download=False)
# ood_loader = torch.utils.data.DataLoader(ood_data, batch_size=64, shuffle=True,
#                                          num_workers=0, pin_memory=False)
# ood_loader.name = "SVHN"
# print('\n\nSVHN Detection')
# extract_features(vits16, ood_loader, 'svhn_features')

# load features

In [ ]:
train_features = torch.load('/content/drive/MyDrive/MQ-solutions/ood_practical/RTL/CIFAR/CIFAR/features224/train_features.pt').cpu().numpy()
in_features = torch.load('/content/drive/MyDrive/MQ-solutions/ood_practical/RTL/CIFAR/CIFAR/features224/test_features.pt').cpu().numpy()

texture_features = torch.load('/content/drive/MyDrive/MQ-solutions/ood_practical/RTL/CIFAR/CIFAR/features224/texture_features.pt').cpu().numpy()
places365_features = torch.load('/content/drive/MyDrive/MQ-solutions/ood_practical/RTL/CIFAR/CIFAR/features224/places365_features.pt').cpu().numpy()
lsun_c_features = torch.load('/content/drive/MyDrive/MQ-solutions/ood_practical/RTL/CIFAR/CIFAR/features224/lsun-c_features.pt').cpu().numpy()
lsun_r_features = torch.load('/content/drive/MyDrive/MQ-solutions/ood_practical/RTL/CIFAR/CIFAR/features224/lsun-r_features.pt').cpu().numpy()
isun_features = torch.load('/content/drive/MyDrive/MQ-solutions/ood_practical/RTL/CIFAR/CIFAR/features224/isun_features.pt').cpu().numpy()
# svhn_features = torch.load('/content/drive/MyDrive/MQ-solutions/ood_practical/RTL/CIFAR/CIFAR/features224/svhn_features.pt')

In [ ]:
train_features.shape, test_features.shape, texture_features.shape, places365_features.shape, lsun_c_features.shape, lsun_r_features.shape, isun_features.shape

((50000, 384),
 (10000, 384),
 (5640, 384),
 (19367, 384),
 (10000, 384),
 (10000, 384),
 (8925, 384))

# model

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet, LinearRegression

class RTL_OOD(object):

    def __init__(self, reduce="none", d=5, norm = 'l2', ood_score = None):
        self.ood_score = ood_score
        self.initial_embed(reduce, d)
        self.initial_norm(norm)
        self.gamma_in = self.gamma_residual
        print(self.__dict__)
        self.scaler = StandardScaler()


    def linear(self, X, y, *args, **kwargs):
        return {"no":y, "linear": self.linear_reranking(X, y, range(len(y)))}
    def linear_reranking(self, X, y = None, index = None):
        self.linear_reg = LinearRegression()
        X_in = X[index]
        y_in = y[index]
        self.linear_reg.fit(X_in, y_in)
        return self.linear_reg.predict(X)

    def reranking(self, X, y = None, percent = 0.6, alpha = 1e-5):
        if y is None:
            y = self.ood_score(X).reshape(-1,1)
        gammas = self.compute_gamma(X, y, alpha)
        index_in = np.argsort(gammas)[:int(len(X) * percent)]
        return self.linear_reranking(X, y, index_in)

    def reranking_list(self, X, y = None, percent = None, alpha = 1e-5):
        if y is None:
            y = self.ood_score(X).reshape(-1,1)
        if not isinstance(percent, list):
            pass
        gammas = self.compute_gamma(X, y, alpha)
        results = {}
        for a_percent in percent:
            index_in = np.argsort(gammas)[:int(len(X) * a_percent)]
            self.linear_reranking(X, y, index_in)
            results[a_percent] = self.linear_reg.predict(X)
        return results

    def compute_gamma(self, X, y = None, alpha = 1e-5):
        if y is None:
            y = self.ood_score(X).reshape(-1,1)
        normed_X = self.norm(X)
        X = self.embed(normed_X)
        H = np.dot(np.dot(X, np.linalg.pinv(np.dot(X.T, X))), X.T)
        X_hat = np.eye(H.shape[0]) - H
        y_hat = np.dot(X_hat, y)
        return self.gamma_in(X_hat, y_hat, alpha)


    def gamma_residual(self, X_hat, y_hat, alpha):

        self.elasticnet = ElasticNet(alpha=alpha, l1_ratio=1.0, fit_intercept=True,
                                #  normalize=True,
                                warm_start=True, selection='cyclic')
        X_hat = self.scaler.fit_transform(X_hat)
        self.elasticnet.fit(X_hat, y_hat)
        print("self.elasticnet.coef_:", self.elasticnet.coef_.shape)
        if self.elasticnet.coef_.ndim == 2:
            #[n_target, n_feature]
            coefs = self.elasticnet.coef_.T
            #[n_feature, n_target] = [n_sample, n_target]
            coefs = np.sum(np.abs(coefs), axis=1)
            return coefs
        elif self.elasticnet.coef_.ndim == 1:
            #[n_feature] = [n_sample]
            coefs = np.abs(self.elasticnet.coef_)
            return coefs
        else:
            raise RuntimeError

    def initial_embed(self, reduce, d):
        reduce = reduce.lower()
        assert reduce in ['isomap', 'ltsa', 'mds', 'lle', 'se', 'pca', 'none']
        if reduce == 'isomap':
            from sklearn.manifold import Isomap
            embed = Isomap(n_components=d)
        elif reduce == 'ltsa':
            from sklearn.manifold import LocallyLinearEmbedding
            embed = LocallyLinearEmbedding(n_components=d,
                                           n_neighbors=5, method='ltsa')
        elif reduce == 'mds':
            from sklearn.manifold import MDS
            embed = MDS(n_components=d, metric=False)
        elif reduce == 'lle':
            from sklearn.manifold import LocallyLinearEmbedding
            embed = LocallyLinearEmbedding(n_components=d, n_neighbors=5,eigen_solver='dense')
        elif reduce == 'se':
            from sklearn.manifold import SpectralEmbedding
            embed = SpectralEmbedding(n_components=d)
        elif reduce == 'pca':
            from sklearn.decomposition import PCA
            embed = PCA(n_components=d)
        if reduce == 'none':
            self.embed = lambda x: x
        else:
            self.embed = lambda x: embed.fit_transform(x)

    def initial_norm(self, norm):
        norm = norm.lower()
        assert norm in ['l2', 'none']
        if norm == 'l2':
            self.norm = lambda x: normalize(x)
        else:
            self.norm = lambda x: x

rtl_ood = RTL_OOD()

{'ood_score': None, 'embed': <function RTL_OOD.initial_embed.<locals>.<lambda> at 0x7e6d654b32e0>, 'norm': <function RTL_OOD.initial_norm.<locals>.<lambda> at 0x7e6d74c2dd00>, 'gamma_in': <bound method RTL_OOD.gamma_residual of <__main__.RTL_OOD object at 0x7e6d747462d0>>}


# utils

In [ ]:
%cd /content/drive/MyDrive/MQ-solutions/ood_practical/RTL/CIFAR
!ls .

/content/drive/.shortcut-targets-by-id/19rDR7CIzlqpEM2y_t3vRDkf0-aHH8tdE/ood_practical/RTL/CIFAR
CIFAR  simul  utils


In [ ]:
from utils.display_results import show_performance, get_measures, print_measures, print_measures_with_std

# fit PyOD

In [ ]:
from pyod.models.auto_encoder import AutoEncoder
from pyod.utils.data import evaluate_print
contamination = 0.001

clf = AutoEncoder(epoch_num=30, contamination=contamination)
clf.fit(train_features)

Training: 100%|██████████| 30/30 [03:13<00:00,  6.46s/it]


In [ ]:
# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

In [ ]:
# get the prediction on the test data
in_pred = clf.predict(in_features)  # outlier labels (0 or 1)
in_score = clf.decision_function(in_features)  # outlier scores

# count 1 and 0 in y_test_pred
np.unique(in_pred, return_counts=True)

(array([0, 1]), array([9992,    8]))

In [ ]:
in_pred.shape, in_scores.shape

((10000,), (10000,))

In [ ]:
# texture
# get the prediction on the test data
y_texture_pred = clf.predict(texture_features)  # outlier labels (0 or 1)
y_texture_scores = clf.decision_function(texture_features)  # outlier scores

# count 1 and 0 in y_texture_pred
np.unique(y_texture_pred, return_counts=True)

(array([1]), array([5640]))

In [ ]:
y_texture_pred.shape, y_texture_scores.shape

((5640,), (5640,))

In [ ]:
ood_num_examples = len(test_features) // 5
expected_ap = ood_num_examples / (ood_num_examples + len(test_features))

In [ ]:
in_score  = in_scores
in_feature = test_features

In [ ]:
def get_ood_scores(features, in_dist=False):
    _score = []
    _feature = []
    _right_score = []
    _wrong_score = []

    if not in_dist:
        features = features[:ood_num_examples]

    preds = clf.predict(features)
    _score = clf.decision_function(features)
    _feature = features

    if in_dist:
        right_indices = preds == 0
        wrong_indices = np.invert(right_indices)
        _right_score.append(_score[-1][right_indices])
        _wrong_score.append(_score[-1][wrong_indices])

        return _score, _right_score, _wrong_score, _feature
    return _score, _feature

In [ ]:
percent = ['no', 'linear']
num_to_avg = 10
alpha = 1e-5

def get_and_print_results(ood_loader):
  aurocs, auprs, fprs = {i:[] for i in percent}, {i:[] for i in percent}, {i:[] for i in percent}

  for _ in range(num_to_avg):
    out_score, out_feature = get_ood_scores(ood_loader)

    num_in = len(in_score)
    print("num_in",num_in)
    in_and_out_score = np.concatenate((in_score, out_score))
    in_and_out_feature = np.concatenate((in_feature, out_feature))
    score_cal = rtl_ood.linear(in_and_out_feature, in_and_out_score, percent, alpha)
    print('successful calculate linear calibration!')

    for a_percent in percent:
      a_score_cal = score_cal[a_percent]
      in_score_changed, out_score = a_score_cal[:num_in], a_score_cal[num_in:]
      measures = get_measures(-in_score_changed, -out_score)

      aurocs[a_percent].append(measures[0]); auprs[a_percent].append(measures[1]); fprs[a_percent].append(measures[2])

  for a_percent in percent:
    auroc = np.mean(aurocs[a_percent])
    aupr = np.mean(auprs[a_percent])
    fpr = np.mean(fprs[a_percent])

    print_measures(auroc, aupr, fpr, a_percent)


# evaluation

In [ ]:
# Texture
get_and_print_results(texture_features)

num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
				no
  FPR95 AUROC AUPR
& 0.00 & 100.00 & 100.00
				linear
  FPR95 AUROC AUPR
& 0.00 & 100.00 & 100.00


In [ ]:
# Places365
get_and_print_results(places365_features)

num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
				no
  FPR95 AUROC AUPR
& 0.00 & 100.00 & 100.00
				linear
  FPR95 AUROC AUPR
& 0.00 & 100.00 & 100.00


In [ ]:
# LSUN-C
get_and_print_results(lsun_c_features)

num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
				no
  FPR95 AUROC AUPR
& 37.55 & 91.61 & 98.01
				linear
  FPR95 AUROC AUPR
& 33.50 & 93.19 & 98.45


In [ ]:
# LSUN-R
get_and_print_results(lsun_r_features)

num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
				no
  FPR95 AUROC AUPR
& 80.00 & 80.58 & 95.65
				linear
  FPR95 AUROC AUPR
& 76.10 & 82.71 & 96.24


In [ ]:
# ISUN
get_and_print_results(isun_features)

num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
num_in 10000
successful calculate linear calibration!
				no
  FPR95 AUROC AUPR
& 75.20 & 82.20 & 96.01
				linear
  FPR95 AUROC AUPR
& 71.70 & 84.33 & 96.60
